In [ ]:
from bs4 import BeautifulSoup as bs
import itertools
import numpy as np
import pandas as pd
import re
import requests
import time

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('max_colwidth', None)

In [ ]:
ump_scores_df = pd.read_csv('D:\Downloads\games (1).csv')

##drop nulls, shortened season
ump_scores_df = ump_scores_df.dropna(axis='index').sort_values('Date')
ump_scores_df = ump_scores_df.set_axis(['date', 'umpire', 'home', 'away', 'home_score', 'away_score', 'called_pitches', 'called_wrong', 'accuracy', 'consistency', 'favor', 'total_run_impact'], axis=1)
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

In [ ]:
##bringing in scraped data from initial scrape (HARD PIVOT - SCRAPE notebook)
day_night_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1)
start_time_df = pd.read_csv('Data\\start_time_df.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1)
duration_df = pd.read_csv('Data\\duration_df.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1)
attendance_df = pd.read_csv('Data\\day_night_df.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1)

##regex cleanup
day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

##set as new columns
ump_scores_df['day_night'] = day_night_df.day_night
ump_scores_df['start_time'] = start_time_df.start_time
ump_scores_df['duration'] = duration_df.duration
ump_scores_df['attendance'] = attendance_df.attendance

In [ ]:
##2nd scrape after fixing LAA 3-letter code to ANA
scrape2_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]

scrape2_df.date = scrape2_df.date.str.replace('-', '')
scrape2_df.loc[scrape2_df.home == 'LAA', 'home'] = 'ANA'
url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
for i, j in zip(scrape2_df.date, scrape2_df.home):
    response = requests.get(url + j + '/' + j + i + '0.shtml')
    soup = bs(response.text, 'html.parser')
    print(j)
    print(i)
    print(response)
    scorebox = soup.find_all('div', {'class':'scorebox_meta'})
    start_time.append(scorebox)
    duration.append(scorebox)
    day_night.append(scorebox)
    time.sleep(3)

In [ ]:
##bringing in scraped data
day_night_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1).set_index(scrape2_df.index)
start_time_df = pd.read_csv('Data\\start_time_df2.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1).set_index(scrape2_df.index)
duration_df = pd.read_csv('Data\\duration_df2.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1).set_index(scrape2_df.index)
attendance_df = pd.read_csv('Data\\day_night_df2.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1).set_index(scrape2_df.index)

##regex cleanup
day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

##set as new columns
scrape2_df['day_night'] = day_night_df.day_night
scrape2_df['start_time'] = start_time_df.start_time
scrape2_df['duration'] = duration_df.duration
scrape2_df['attendance'] = attendance_df.attendance
scrape2_df

##update main dataframe
ump_scores_df.update(scrape2_df)

In [ ]:
##3rd scrape to bring in start_time, duration, day/night, and attendance data
scrape3_df = ump_scores_df.loc[ump_scores_df.start_time.isnull()]
scrape3_df = scrape3_df.sort_values(['date', 'home'])
scrape3_df.date = scrape3_df.date.str.replace('-', '')
scrape3_df.loc[scrape3_df.home == 'LAA', 'home'] = 'ANA'
scrape3_df.loc[scrape3_df.home == 'WSH', 'home'] = 'WAS'
scrape3_df.loc[scrape3_df.home == 'LAD', 'home'] = 'LAN'
scrape3_df.loc[scrape3_df.home == 'SD', 'home'] = 'SDN'
scrape3_df.loc[scrape3_df.home == 'SF', 'home'] = 'SFN'
scrape3_df.loc[scrape3_df.home == 'NYY', 'home'] = 'NYA'
scrape3_df.loc[scrape3_df.home == 'KC', 'home'] = 'KCA'
scrape3_df.loc[scrape3_df.home == 'STL', 'home'] = 'SLN'
scrape3_df.loc[scrape3_df.home == 'NYM', 'home'] = 'NYN'
scrape3_df.loc[scrape3_df.home == 'TB', 'home'] = 'TBA'
scrape3_df.loc[scrape3_df.home == 'CWS', 'home'] = 'CHA'
scrape3_df.loc[scrape3_df.home == 'CHC', 'home'] = 'CHN'

url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
n = 0
for i, j in zip(scrape3_df.date, scrape3_df.home):
    n += 1

    if n%2 != 0:
        response = requests.get(url + j + '/' + j + i + '1.shtml')
        soup = bs(response.text, 'html.parser')
        scorebox = soup.find_all('div', {'class':'scorebox_meta'})
        start_time.append(scorebox)
        duration.append(scorebox)
        day_night.append(scorebox)
        time.sleep(3)

    if n%2 == 0:
        response = requests.get(url + j + '/' + j + i + '2.shtml')
        soup = bs(response.text, 'html.parser')
        scorebox = soup.find_all('div', {'class':'scorebox_meta'})
        start_time.append(scorebox)
        duration.append(scorebox)
        day_night.append(scorebox)
        time.sleep(3)

In [ ]:
##set list of lists to single string by converting to dataframe, saving as csv, and bringing in csv as dataframe
start_time_df = pd.DataFrame(start_time)
duration_df = pd.DataFrame(duration)
day_night_df = pd.DataFrame(day_night)

start_time_df.to_csv('start_time_df3.csv')
duration_df.to_csv('duration_df3.csv')
day_night_df.to_csv('day_night_df3.csv')

day_night_df = pd.read_csv('day_night_df3.csv').drop(columns='Unnamed: 0').set_axis(['day_night'], axis=1).set_index(scrape3_df.index)
start_time_df = pd.read_csv('start_time_df3.csv').drop(columns='Unnamed: 0').set_axis(['start_time'], axis=1).set_index(scrape3_df.index)
duration_df = pd.read_csv('duration_df3.csv').drop(columns='Unnamed: 0').set_axis(['duration'], axis=1).set_index(scrape3_df.index)
attendance_df = pd.read_csv('day_night_df3.csv').drop(columns='Unnamed: 0').set_axis(['attendance'], axis=1).set_index(scrape3_df.index)

day_night_df.day_night = day_night_df.day_night.str.extract(r'((?<=\d\:\d{2}</div><div>).*?(?=</div><div>))')
start_time_df.start_time = start_time_df.start_time.str.extract(r'((?<=Time: ).*?(?= Local))')
duration_df.duration = duration_df.duration.str.extract(r'((?<=n</strong>: ).*?(?=</div>))')
attendance_df.attendance = attendance_df.attendance.str.extract(r'((?<=ce</strong>: ).*?(?=</div>))')

scrape3_df['day_night'] = day_night_df.day_night
scrape3_df['start_time'] = start_time_df.start_time
scrape3_df['duration'] = duration_df.duration
scrape3_df['attendance'] = attendance_df.attendance

##manual cleanup of 2 games
scrape3_df.home[6391] = 'MIL'
scrape3_df.away[6391] = 'MIA'
scrape3_df.day_night[6391] = 'Night Game, on grass'
scrape3_df.start_time[6391] = '7:11 p.m.'
scrape3_df.duration[6391] = '3:23'
scrape3_df.attendance[6391] = '19,369'
scrape3_df.home[6421] = 'MIL'
scrape3_df.away[6421] = 'MIA'
scrape3_df.day_night[6421] = 'Night Game, on grass'
scrape3_df.start_time[6421] = '6:10 p.m.'
scrape3_df.duration[6421] = '3:03'
scrape3_df.attendance[6421] = '25,079'
scrape3_df

##update into main dataframe
ump_scores_df.update(scrape3_df)

In [ ]:
##sanity check to find incomplete values
##manual cleanup afterwards of a few games
sanity_check_df = scrape3_df.loc[[305, 323, 337, 2024, 5302, 6109, 6235, 6329, 6776, 6808, 6862, 6900, 7230, 7326, 7829, 8005, 8145, 8446, 9454, 10315, 12714, 12726]]
sanity_check_df.day_night[[305, 323, 337]] = ['Night Game, on turf', 'Night Game, on turf', 'Day Game, on turf']
sanity_check_df.start_time[[305, 323, 337]] = ['7:08 p.m.', '7:06 p.m.', '1:37 p.m.']
sanity_check_df.duration[[305, 323, 337]] = ['2:19', '2:33', '2:44']
sanity_check_df.attendance[[305, 323, 337]] = ['9,945', '12,789', '16,652']
sanity_check_df.home[12726] = 'ANA'
sanity_check_df.away[12726] = 'TOR'
sanity_check_df.day_night[12726] = sanity_check_df.day_night[12714]
sanity_check_df.start_time[12726] = sanity_check_df.start_time[12714]
sanity_check_df.duration[12726] = sanity_check_df.duration[12714]
sanity_check_df.attendance[12726] = sanity_check_df.attendance[12714]
sanity_check_df.day_night[12714] = 'Night Game, on grass'
sanity_check_df.start_time[12714] = '7:07 p.m.'
sanity_check_df.duration[12714] = '2:23'
sanity_check_df.attendance[12714] = '14,443'
sanity_check_df

##update into main dataframe
ump_scores_df.update(sanity_check_df)

In [ ]:
##for loop to determine if games were played during the day or night
##this was necessary as for doubleheaders this data was missing
umptime_list = list(ump_scores_df.start_time.unique())
umptime_list

daytime_list = ['11:', '12:', '1:', '2:', '3:', '4:']
nighttime_list = ['5:', '6:', '7:', '8:', '9:', '10:']

daygame_list = []
nightgame_list = []
for i in umptime_list:
    for j in daytime_list:
        if j in i:
            daygame_list.append(i)
    for j in nighttime_list:
        if j in i:
            nightgame_list.append(i)

n = 0
for i in ump_scores_df.start_time:
    for j in daygame_list:
        if i == j:
            ump_scores_df.at[n, 'day_night'] = 'Day'
            ump_scores_df.at[n, 'date'] = ump_scores_df.at[n, 'date'].replace('-', '')
    for j in nightgame_list:
        if i == j:
            ump_scores_df.at[n, 'day_night'] = 'Night'
            ump_scores_df.at[n, 'date'] = ump_scores_df.at[n, 'date'].replace('-', '')
    n += 1

##attendance of game at index 285 was actually 0 due to Baltimore riots
ump_scores_df.at[285, 'attendance'] = 0

In [ ]:
##group umpire and view mean accuracy/consistency
scatterplot_df = pd.DataFrame()
scatterplot_df[['accuracy', 'consistency']] = ump_scores_df.groupby('umpire')[['accuracy', 'consistency']].mean()
scatterplot_df['games'] = ump_scores_df.groupby('umpire')['accuracy'].count()
scatterplot_df.to_csv('scatterplot_df.csv')

In [ ]:
##group umpire and view mean accuracy/consistency by day and night
umpscores_day_df = ump_scores_df.loc[ump_scores_df.day_night == 'Day'].reset_index(drop=True)
umpscores_day_plot_df = umpscores_day_df.groupby('umpire')[['accuracy', 'consistency']].mean()
umpscores_day_plot_df['games'] = umpscores_day_df.groupby('umpire')['accuracy'].count()

umpscores_night_df = ump_scores_df.loc[ump_scores_df.day_night == 'Night'].reset_index(drop=True)
umpscores_night_plot_df = umpscores_night_df.groupby('umpire')[['accuracy', 'consistency']].mean()
umpscores_night_plot_df['games'] = umpscores_night_df.groupby('umpire')['accuracy'].count()

umpscores_diff_df = (umpscores_day_plot_df[['accuracy', 'consistency']] - umpscores_night_plot_df[['accuracy', 'consistency']])
umpscores_diff_df[['day_games', 'night_games']] = [umpscores_day_plot_df.games, umpscores_night_plot_df.games]

umpscores_day_plot_df.to_csv('umpscores_day_plot_df.csv')
umpscores_night_plot_df.to_csv('umpscores_night_plot_df.csv')
umpscores_diff_df.to_csv('umpscores_diff_df.csv')

In [ ]:
##for loop to pull umpire birthyear
ump_list = list(ump_scores_df.umpire.unique())
ump_list = [i.replace(' ','_') for i in ump_list]
ump_list

url = 'https://www.baseball-reference.com/bullpen/'

ump_birthyear = []
n = 0
for i in ump_list:
    response = requests.get(url + str(i))
    soup = bs(response.text, 'html.parser')
    ump_birthyear.append(soup.findAll('a'))
    time.sleep(3)
    n += 1
    print(n)
    print(url+str(i))

In [ ]:
ump_birthyear_df = pd.DataFrame(ump_birthyear)
ump_birthyear_df.to_csv('ump_birthyear.csv')
ump_birthyear_df = pd.read_csv('ump_birthyear.csv').drop(columns='Unnamed: 0').applymap(str).apply(' '.join, axis=1)
ump_birthyear_df = pd.DataFrame(ump_birthyear_df).set_axis(['birthyear'], axis=1)

##manual cleanup of a few birthyears that were null
ump_birthyear_df.birthyear = ump_birthyear_df.birthyear.str.extract(r'((?<=title=\"\d{4}\">).*?(?=</a>))')
ump_birthyear_df.birthyear[6] = 1955
ump_birthyear_df.birthyear[85] = 1986
ump_birthyear_df.birthyear[98] = 1984
ump_birthyear_df.birthyear[100] = 1988
ump_birthyear_df.birthyear[102] = 1992
ump_birthyear_df.birthyear[103] = 1985
ump_birthyear_df.birthyear[111] = 1990
ump_birthyear_df.birthyear[115] = 1985

ump_list = [i.replace('_',' ') for i in ump_list]
ump_birthyear_df['umpire'] = ump_list

In [ ]:
##pulled year/month/day from gamedate in order for it to easier to work with
ump_scores_df['date_year'] = ump_scores_df['date'].str[:4]
ump_scores_df['date_month'] = ump_scores_df['date'].str[4:6]
ump_scores_df['date_day'] = ump_scores_df['date'].str[6:]
ump_scores_df = ump_scores_df.merge(ump_birthyear_df, how='left')

##input correct birthyear for 2 umpires that were incorrect
ump_scores_df.loc[ump_scores_df.birthyear == 1941, 'birthyear'] = 1984
ump_scores_df.loc[ump_scores_df.birthyear == 2009, 'birthyear'] = 1978

##create age column
ump_scores_df['umpire_age'] = ump_scores_df.date_year.astype(int) - ump_scores_df.birthyear.astype(int)
ump_scores_df.birthyear = ump_scores_df.birthyear.astype(int)

In [ ]:
##group age and view mean accuracy/consistency
age_stats = pd.DataFrame(ump_scores_df.groupby('umpire_age')[['accuracy', 'consistency']].mean())
age_stats['games'] = ump_scores_df.groupby('umpire_age')['accuracy'].count()
age_stats.to_csv('age_stats.csv')

In [ ]:
##for loop to bin game duration
duration_bin = []
for i in ump_scores_df.duration:
    if '1:' in i:
        duration_bin.append('1:00-1:59')
    elif '2:' in i:
        duration_bin.append('2:00-2:59')
    elif '3:' in i:
        duration_bin.append('3:00-3:59')
    elif '4:' in i:
        duration_bin.append('4:00-4:59')
    elif '5:' in i:
        duration_bin.append('5:00-5:59')
    elif '6:' in i:
        duration_bin.append('6:00-6:59')
    elif '7:' in i:
        duration_bin.append('7:00-7:59')
    else:
        print('WHAT HAPPENED')

ump_scores_df['duration_bin'] = duration_bin

In [ ]:
##group duration bins and view mean accuracy/consistency with total games
duration_stats = pd.DataFrame(ump_scores_df.groupby('duration_bin')[['accuracy', 'consistency']].mean())
duration_stats['games'] = ump_scores_df.groupby('duration_bin')['accuracy'].count()
duration_stats.to_csv('duration_stats.csv')

In [ ]:
##for loop to bin called pitches
calledpitch_bin = []
for i in ump_scores_df.called_pitches:
    if   str(i).startswith(('6','7','8','9')):
        calledpitch_bin.append('0-100')
    elif str(i).startswith(('10','11')):
        calledpitch_bin.append('100-119')
    elif str(i).startswith(('12','13')):
        calledpitch_bin.append('120-139')
    elif str(i).startswith(('14','15')):
        calledpitch_bin.append('140-159')
    elif str(i).startswith(('16','17')):
        calledpitch_bin.append('160-179')
    elif str(i).startswith(('18','19')):
        calledpitch_bin.append('180-199')
    elif str(i).startswith(('20','21')):
        calledpitch_bin.append('200-219')
    elif str(i).startswith(('22','23')):
        calledpitch_bin.append('220-239')
    elif str(i).startswith(('24','25')):
        calledpitch_bin.append('240-259')
    elif str(i).startswith(('26','27')):
        calledpitch_bin.append('260-279')
    elif str(i).startswith(('28','29')):
        calledpitch_bin.append('280-299')
    else:
        calledpitch_bin.append('300+')

ump_scores_df['calledpitch_bin'] = calledpitch_bin

In [ ]:
##group called pitch bins and view mean accuracy/consistency
calledpitch_stats = pd.DataFrame(ump_scores_df.groupby('calledpitch_bin')[['accuracy', 'consistency']].mean())
calledpitch_stats['games'] = ump_scores_df.groupby('calledpitch_bin')['accuracy'].count()
calledpitch_stats.to_csv('calledpitch_stats.csv')

In [ ]:
ump_scores_df.to_csv('ump_scores_df.csv')